## IMPLEMENTATION OF NEURAL-PROPHET

##### Libraries and Modules

In [ ]:
import numpy as np
import pandas as pd
import itertools
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.graph_objects as go

from datetime import datetime
from neuralprophet import NeuralProphet
from neuralprophet import set_random_seed
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [ ]:
# Load the original data
processed_data = pd.read_excel('../Data/df_time_series.xlsx')

In [ ]:
# Convert column 'DATE' to datetime type and set it as index
processed_data['FECHA'] = pd.to_datetime(processed_data['FECHA'])
# Set the index
processed_data.set_index('FECHA', inplace=True)
# Create a date range with a frequency of 15 days and set it as an index
processed_data = processed_data.asfreq('15D')

In [ ]:
# List of columns to process
columns = ['CLORURO DE SODIO', 'LACTATO DE SODIO', 'METAMIZOL', 'PARACETAMOL', 'PROPOFOL']

In [ ]:
# Iterate over each column
for column in columns:
    # Create the DataFrame for NeuralProphet
    df = pd.DataFrame({
        'ds': processed_data.index,
        'y': processed_data[column]
    })

    # Split data into training and validation sets
    train_size = int(len(df) * 0.8)  # 80% for training
    train_df = df[:train_size]
    valid_df = df[train_size:]

    # Model and prediction with custom 15-day seasonality
    m = NeuralProphet(
        # Disable trend changepoints
        n_changepoints=0,
        # Disable other seasonality components
        yearly_seasonality=False,
        weekly_seasonality=False,
        daily_seasonality=False,
        # Add custom 15-day seasonality
        seasonality_mode='additive',  # or 'multiplicative' based on your data
        seasonality_reg=0.1,  # You can adjust regularization strength
        n_lags=15,  # Number of lags for 15-day seasonality
    )
    m.set_plotting_backend("plotly")

    # Fit the model with training data
    metrics = m.fit(train_df)

    # Predict on the validation set
    forecast = m.predict(valid_df)

    # Adjust forecast values below 0 to 0
    forecast['yhat1'] = forecast['yhat1'].clip(lower=0)

    # Plot the forecast along with the original data and the training portion
    fig = m.plot(forecast)
    fig.add_scatter(x=df['ds'], y=df['y'], mode='lines', name='Original Data')
    fig.add_scatter(x=train_df['ds'], y=train_df['y'], mode='lines', name='Training Data')

    # Modify the figure, change the marker type for actual values
    fig.update_traces(mode='lines')  # You can customize size and symbol as needed
    
    # Show the modified plot
    fig.show()

    print(f"Results for column: {column}")
    print(metrics)
